In [ ]:
import pickle
import numpy as np
from collections import defaultdict

# --- USER SETTING ---
PERCENTILE = 75  # flag all windows above this percentile (e.g. 95th percentile)

# --- Load Results ---
with open('ensemble_res.pkl', 'rb') as f:
    all_results = pickle.load(f)

# --- Compute Anomaly Scores ---
anomaly_scores = defaultdict(lambda: {'score_sum': 0.0, 'count': 0})

for iteration_result in all_results:
    buckets = iteration_result['buckets']
    bucket_results = iteration_result['bucket_results']

    for bucket_result in bucket_results:
        bucket_idx = bucket_result['bucket_idx']
        final_results = bucket_result['final_results']
        indices_in_bucket = buckets[bucket_idx]

        mean = np.mean(final_results)
        std = np.std(final_results) if np.std(final_results) != 0 else 1e-8

        for i, idx in enumerate(indices_in_bucket):
            sim = final_results[i]
            deviation = abs(sim - mean) / std
            anomaly_scores[idx]['score_sum'] += deviation
            anomaly_scores[idx]['count'] += 1

# --- Final Score Calculation ---
final_scores = {
    idx: score_data['score_sum'] / score_data['count']
    for idx, score_data in anomaly_scores.items()
}

# --- Compute Threshold by Percentile ---
all_score_values = list(final_scores.values())
threshold = np.percentile(all_score_values, PERCENTILE)

# --- Flag Anomalous Windows ---
anomalous_windows = {
    idx: score for idx, score in final_scores.items() if score >= threshold
}

print(f"\n{len(anomalous_windows)} anomalous windows detected above the {PERCENTILE}th percentile (threshold ≈ {threshold:.4f}):\n")
for idx, score in sorted(anomalous_windows.items()):
    print(f"Window {idx} - Score: {score:.4f}")

# --- Save to file ---
with open("flagged_anomalous_windows.txt", "w") as f:
    for idx, score in sorted(anomalous_windows.items()):
        f.write(f"Window {idx} - Score: {score:.4f}\n")

print("\nSaved to 'flagged_anomalous_windows.txt'")



74 anomalous windows detected above the 10th percentile (threshold ≈ 0.8304):

Window 0 - Score: 0.8689
Window 1 - Score: 0.8921
Window 2 - Score: 0.8918
Window 3 - Score: 0.9045
Window 4 - Score: 0.8682
Window 5 - Score: 0.8677
Window 6 - Score: 0.8791
Window 7 - Score: 0.8774
Window 8 - Score: 0.8708
Window 9 - Score: 0.8370
Window 11 - Score: 0.8489
Window 13 - Score: 0.8600
Window 14 - Score: 0.8349
Window 15 - Score: 0.8491
Window 18 - Score: 0.9016
Window 19 - Score: 0.8558
Window 20 - Score: 0.8711
Window 21 - Score: 0.8920
Window 22 - Score: 0.8750
Window 23 - Score: 0.8451
Window 24 - Score: 0.8314
Window 25 - Score: 0.8983
Window 27 - Score: 0.8696
Window 28 - Score: 0.8758
Window 30 - Score: 0.8771
Window 31 - Score: 0.8864
Window 32 - Score: 0.8592
Window 34 - Score: 0.8450
Window 35 - Score: 0.8524
Window 36 - Score: 0.8862
Window 37 - Score: 0.8561
Window 38 - Score: 0.8773
Window 39 - Score: 0.8814
Window 40 - Score: 0.8626
Window 41 - Score: 0.9205
Window 42 - Score: 0